In [3]:
!pip install ezdxf


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 5.6 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 8.4 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -------------------- ------------------- 1/2 [ezdxf]
   -----

In [1]:
import ezdxf

# --- Load DXF File ---
def load_dxf(file_path):
    try:
        doc = ezdxf.readfile(file_path)
        print(f"DXF loaded successfully: {file_path}")
        return doc
    except IOError:
        print(f"File not found: {file_path}")
    except ezdxf.DXFStructureError:
        print("Invalid or corrupted DXF file.")
    return None

# --- Explore Modelspace Entities ---
def list_modelspace_entities(doc):
    msp = doc.modelspace()
    print("\n--- Entities in Modelspace ---")
    for e in msp:
        print(f"{e.dxftype():<10} - Handle: {e.dxf.handle} - Layer: {e.dxf.layer}")

# --- Access Basic Entity Types ---
def show_entity_examples(doc):
    msp = doc.modelspace()

    print("\n--- Lines ---")
    for line in msp.query('LINE'):
        print(f"Start: {line.dxf.start}, End: {line.dxf.end}, Layer: {line.dxf.layer}")

    print("\n--- Texts ---")
    for text in msp.query('TEXT'):
        print(f"Text: '{text.dxf.text}', Insert Point: {text.dxf.insert}, Height: {text.dxf.height}")

    print("\n--- MTEXTs ---")
    for mtext in msp.query('MTEXT'):
        print(f"MTEXT: '{mtext.text}', Insert Point: {mtext.dxf.insert}")

# --- List All Blocks and Block References ---
def list_blocks(doc):
    print("\n--- Defined Blocks ---")
    for block in doc.blocks:
        print(f"Block Name: {block.name}, Entities: {len(block)}")

    msp = doc.modelspace()
    print("\n--- Block References (INSERT) ---")
    for insert in msp.query('INSERT'):
        print(f"BlockRef: {insert.dxf.name}, Insert Point: {insert.dxf.insert}, Layer: {insert.dxf.layer}")
        # Optional: show attribute texts inside the block
        for attrib in insert.attribs:
            print(f"    ATTRIB: Tag = {attrib.dxf.tag}, Text = '{attrib.dxf.text}'")

# --- Main Test Function for Jupyter ---
def test_dxf_parsing(file_path):
    doc = load_dxf(file_path)
    if doc:
        list_modelspace_entities(doc)
        show_entity_examples(doc)
        list_blocks(doc)


In [2]:
test_dxf_parsing("input.dxf")

DXF loaded successfully: input.dxf

--- Entities in Modelspace ---
INSERT     - Handle: 7A - Layer: 0
INSERT     - Handle: 7B - Layer: pole
INSERT     - Handle: 7C - Layer: pole
INSERT     - Handle: 7D - Layer: pole
INSERT     - Handle: 7E - Layer: 0
INSERT     - Handle: 7F - Layer: 0
INSERT     - Handle: 80 - Layer: 0
INSERT     - Handle: 81 - Layer: pole
INSERT     - Handle: 82 - Layer: pole
LINE       - Handle: 83 - Layer: wire_3PH_primary
LINE       - Handle: 84 - Layer: wire_3PH_primary
LINE       - Handle: 85 - Layer: wire_3PH_primary
LINE       - Handle: 86 - Layer: wire_3PH_primary
INSERT     - Handle: 87 - Layer: pole
INSERT     - Handle: 88 - Layer: 0
LINE       - Handle: 89 - Layer: wire_secondary
LINE       - Handle: 8A - Layer: wire_service
LINE       - Handle: 8B - Layer: wire_service
LINE       - Handle: 8C - Layer: wire_2PH_primary
LINE       - Handle: 8D - Layer: wire_Z_primary
LINE       - Handle: 8E - Layer: wire_Y_primary
LINE       - Handle: 8F - Layer: wire_X_prim

In [22]:
import ezdxf
design = load_dxf("Sample1.dxf")

DXF loaded successfully: Sample1.dxf


# Pick one pole to work on

In [23]:
# Find and return the first INSERT entity on layer "pole"
def get_first_pole(doc):
    msp = doc.modelspace()
    for insert in msp.query('INSERT'):
        if insert.dxf.layer.lower() == 'pole':
            return insert
    return None


In [24]:
pole = get_first_pole(design)

In [1]:
# Read only
import ezdxf

def replace_pole_with_library(doc, output_path):
    msp = doc.modelspace()
    
    # 1. Find the first pole on layer "pole"
    poles = msp.query('INSERT[layer=="pole"]')
    pole = next(iter(poles), None)
    if not pole:
        print("No pole found on layer 'pole'.")
        return

    ins_point = pole.dxf.insert
    pole.destroy()  # remove the original pole

    # 2. Load and insert "Removed Pole"
    removed_doc = ezdxf.readfile("overhead/pole/Removed Pole.dxf")
    removed_block = doc.blocks.new(name="Removed_Pole_Block")
    for e in removed_doc.modelspace():
        removed_block.add_entity(e.copy())
    msp.add_blockref("Removed_Pole_Block", ins_point, dxfattribs={"layer": "pole"})

    # 3. Load and insert "Replaced Pole"
    replaced_doc = ezdxf.readfile("overhead/pole/Replaced Pole.dxf")
    replaced_block = doc.blocks.new(name="Replaced_Pole_Block")
    for e in replaced_doc.modelspace():
        replaced_block.add_entity(e.copy())
    msp.add_blockref("Replaced_Pole_Block", ins_point, dxfattribs={"layer": "pole"})

    # 4. Save the modified file
    doc.saveas(output_path)
    print(f"Saved modified DXF to {output_path}")

# Usage example:
doc = ezdxf.readfile("input.dxf")
replace_pole_with_library(doc, "output.dxf")


DXFTableEntryError: BLOCK_RECORD 'Removed_Pole_Block' already exists!

In [39]:
import math
import ezdxf
from ezdxf.math import Vec2

primary_line_layers = ['wire_3PH_primary', 'wire_X_primary', 'wire_Y_primary', 'wire_Z_primary', 'wire_2PH_primary']

def is_close(p1, p2, tol=1):
    return Vec2(p1).distance(Vec2(p2)) <= tol

def add_text_on_line(msp, start, end, text="RP", color=6, height=17):
    midpoint = Vec2(start).lerp(Vec2(end), factor=0.5)
    angle_rad = math.atan2(end[1] - start[1], end[0] - start[0])
    angle_deg = math.degrees(angle_rad)

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    # ✅ Set alignment to center on midpoint
    txt.dxf.insert = midpoint
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = midpoint


def add_length_label_below(msp, start, end, color=6, height= 13, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)
    
    length = start_v.distance(end_v)
    length_text = f"{length:.2f}"
    
    # Calculate angle of line in radians
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)
    
    # Vector pointing along the line
    line_dir = (end_v - start_v).normalize()
    # Perpendicular vector to the line (rotated 90 deg CCW)
    perp_dir = Vec2(-line_dir.y, line_dir.x)
    
    # Always offset downward visually means offset along negative perp_dir
    text_pos = midpoint - perp_dir * offset_distance * 20
    
    # Add the text
    txt = msp.add_text(
        length_text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos
    
# Can be used to put a text above line
def add_text_above_line(msp, start, end, text, color=6, height=10, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)

    # Angle of the line for text rotation
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)

    # Direction perpendicular to the line (90 degrees CCW)
    line_dir = (end_v - start_v).normalize()
    perp_dir = Vec2(-line_dir.y, line_dir.x)

    # Move in the **positive** perpendicular direction → above visually
    text_pos = midpoint + perp_dir * offset_distance * 15

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos


def replace_pole_with_library(doc, output_path):
    msp = doc.modelspace()

    # Load and define Removed_Pole_Block once
    if "Removed_Pole_Block" not in doc.blocks:
        removed_doc = ezdxf.readfile("overhead/pole/Removed Pole.dxf")
        removed_block = doc.blocks.new(name="Removed_Pole_Block")
        for e in removed_doc.modelspace():
            removed_block.add_entity(e.copy())

    # Load and define Replaced_Pole_Block once
    if "Replaced_Pole_Block" not in doc.blocks:
        replaced_doc = ezdxf.readfile("overhead/pole/Replaced Pole.dxf")
        replaced_block = doc.blocks.new(name="Replaced_Pole_Block")
        for e in replaced_doc.modelspace():
            replaced_block.add_entity(e.copy())

    if "Ground" not in doc.blocks:
        # Load and define the Ground block if not already loaded
        ground_doc = ezdxf.readfile("overhead/ground/Ground.dxf")
        ground_block = doc.blocks.new(name="Ground")
        for e in ground_doc.modelspace():
            ground_block.add_entity(e.copy())

    # Loop through wires
    for line in msp.query('LINE'):
        if line.dxf.layer in primary_line_layers:
            # Replace the connecting poles (It makes sure that the poles are primary)
            for pole in msp.query('INSERT[name ? "^Pole.*"]'):
                pole_pos = pole.dxf.insert
                if is_close(pole_pos, line.dxf.start) or is_close(pole_pos, line.dxf.end):
                    ins_point = pole.dxf.insert
                    pole.destroy()  # Remove old pole
                    # Insert new pole
                    msp.add_blockref("Removed_Pole_Block", ins_point, dxfattribs={"layer": "pole"})
                    msp.add_blockref("Replaced_Pole_Block", ins_point, dxfattribs={"layer": "pole"})
                    # Insert Ground
                    msp.add_blockref("Ground", ins_point, dxfattribs={"layer": "ground"})
    
            # future use for layer based selection
            # if line.dxf.layer == "wire_primary"
    
            # Change wire to replace notation
            add_text_on_line(msp, line.dxf.start, line.dxf.end)
            add_length_label_below(msp, line.dxf.start, line.dxf.end, offset_distance=1)
            add_text_above_line(msp, line.dxf.start, line.dxf.end, "This is test", offset_distance=1.5)

    pole_text = "pole_text"
    for txt in msp.query(f'*[layer=="{pole_text}"]'):
        old_content = txt.dxf.text
        new_content = f"{old_content}\n45ft-A"
        txt.dxf.text = new_content
        
    doc.saveas(output_path)
    print(f"Saved modified DXF to {output_path}")


# Usage example:
doc = ezdxf.readfile("input.dxf")
replace_pole_with_library(doc, "output.dxf")


Saved modified DXF to output.dxf


In [43]:
import math
import ezdxf
from ezdxf.math import Vec2

primary_line_layers = ['wire_3PH_primary', 'wire_X_primary', 'wire_Y_primary', 'wire_Z_primary', 'wire_2PH_primary']

def is_close(p1, p2, tol=1):
    return Vec2(p1).distance(Vec2(p2)) <= tol

def add_text_on_line(msp, start, end, text="RP", color=6, height=17):
    midpoint = Vec2(start).lerp(Vec2(end), factor=0.5)
    angle_rad = math.atan2(end[1] - start[1], end[0] - start[0])
    angle_deg = math.degrees(angle_rad)

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    # ✅ Set alignment to center on midpoint
    txt.dxf.insert = midpoint
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = midpoint


def add_length_label_below(msp, start, end, color=6, height= 13, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)
    
    length = start_v.distance(end_v)
    length_text = f"{length:.2f}"
    
    # Calculate angle of line in radians
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)
    
    # Vector pointing along the line
    line_dir = (end_v - start_v).normalize()
    # Perpendicular vector to the line (rotated 90 deg CCW)
    perp_dir = Vec2(-line_dir.y, line_dir.x)
    
    # Always offset downward visually means offset along negative perp_dir
    text_pos = midpoint - perp_dir * offset_distance * 20
    
    # Add the text
    txt = msp.add_text(
        length_text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos
    
# Can be used to put a text above line
def add_text_above_line(msp, start, end, text, color=6, height=10, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)

    # Angle of the line for text rotation
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)

    # Direction perpendicular to the line (90 degrees CCW)
    line_dir = (end_v - start_v).normalize()
    perp_dir = Vec2(-line_dir.y, line_dir.x)

    # Move in the **positive** perpendicular direction → above visually
    text_pos = midpoint + perp_dir * offset_distance * 15

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos


def replace_pole_with_library(doc, output_path):
    msp = doc.modelspace()

    # Load and define Removed_Pole_Block once
    if "Removed_Pole_Block" not in doc.blocks:
        removed_doc = ezdxf.readfile("overhead/pole/Removed Pole.dxf")
        removed_block = doc.blocks.new(name="Removed_Pole_Block")
        for e in removed_doc.modelspace():
            removed_block.add_entity(e.copy())

    # Load and define Replaced_Pole_Block once
    if "Replaced_Pole_Block" not in doc.blocks:
        replaced_doc = ezdxf.readfile("overhead/pole/Replaced Pole.dxf")
        replaced_block = doc.blocks.new(name="Replaced_Pole_Block")
        for e in replaced_doc.modelspace():
            replaced_block.add_entity(e.copy())

    # Load and define Ground block once
    if "Ground" not in doc.blocks:
        ground_doc = ezdxf.readfile("overhead/ground/Ground.dxf")
        ground_block = doc.blocks.new(name="Ground")
        for e in ground_doc.modelspace():
            ground_block.add_entity(e.copy())

    # Query existing Ground inserts on "overhead_device" layer for quick lookup
    existing_grounds = [e for e in msp.query('INSERT') if e.dxf.layer == "overhead_device" and e.dxf.name == "Ground"]

    # Loop through wires
    for line in msp.query('LINE'):
        if line.dxf.layer in primary_line_layers:
            for pole in msp.query('INSERT[name ? "^Pole.*"]'):
                pole_pos = pole.dxf.insert
                if is_close(pole_pos, line.dxf.start) or is_close(pole_pos, line.dxf.end):
                    ins_point = pole_pos
                    pole.destroy()  # Remove old pole

                    # Insert Removed and Replaced Poles (on "pole" layer)
                    msp.add_blockref("Removed_Pole_Block", ins_point, dxfattribs={"layer": "pole"})
                    msp.add_blockref("Replaced_Pole_Block", ins_point, dxfattribs={"layer": "pole"})

                    # Check if Ground block already exists at pole position (within tolerance)
                    ground_exists = False
                    for ground in existing_grounds:
                        if is_close(ins_point, ground.dxf.insert):
                            ground_exists = True
                            break

                    if not ground_exists:
                        # Insert Ground block scaled 4x on "overhead_device" layer
                        msp.add_blockref(
                            "Ground",
                            ins_point,
                            dxfattribs={
                                "layer": "overhead_device",
                                "xscale": 3.5,
                                "yscale": 3.5,
                            }
                        )

            # Add texts and labels
            add_text_on_line(msp, line.dxf.start, line.dxf.end)
            add_length_label_below(msp, line.dxf.start, line.dxf.end, offset_distance=1)
            add_text_above_line(msp, line.dxf.start, line.dxf.end, "This is test", offset_distance=1.5)

    # Update texts under pole_text layer
    pole_text = "pole_text"
    for txt in msp.query(f'*[layer=="{pole_text}"]'):
        old_content = txt.dxf.text
        new_content = f"{old_content}\n45ft-A"
        txt.dxf.text = new_content

    doc.saveas(output_path)
    print(f"Saved modified DXF to {output_path}")


# Usage example:
doc = ezdxf.readfile("input.dxf")
replace_pole_with_library(doc, "output.dxf")


Saved modified DXF to output.dxf


In [47]:
import math
import ezdxf
from ezdxf.math import Vec2

primary_line_layers = [
    'wire_3PH_primary', 'wire_X_primary',
    'wire_Y_primary', 'wire_Z_primary', 'wire_2PH_primary'
]

def is_close(p1, p2, tol=1):
    return Vec2(p1).distance(Vec2(p2)) <= tol

def install_down_guys(msp, pole_pos, wire_dirs, down_guy_block_name="Down_Guy", scale=1.0):
    """
    Remove existing down-guys at this pole, then install new ones to balance tension.
    wire_dirs: list of Vec2 unit-vectors pointing along each primary wire leaving the pole.
    """
    # 1) Remove old DownGuy blocks at this pole
    for guy in msp.query('INSERT[name=="DownGuy"]'):
        if guy.dxf.layer == "overhead_device" and is_close(pole_pos, guy.dxf.insert):
            guy.destroy()

    # 2) Compute equilibrium directions
    installs = []  # list of unit Vec2 directions where we'll place a down-guy
    n = len(wire_dirs)
    if n == 2:
        # two wires: if straight (180°), single guy opposite; else two guys opposite each wire
        angle = math.degrees(math.acos(wire_dirs[0].dot(wire_dirs[1])))
        if abs(angle - 180) < 1e-3:
            installs = [ -(wire_dirs[0] + wire_dirs[1]).normalize() ]
        else:
            installs = [ -d for d in wire_dirs ]
    elif n == 1:
        # one wire → place opposite it
        installs = [ -wire_dirs[0] ]
    else:
        # three or more: weight by tension (t for 3-phase, t/3 for singles) → net tension vector
        # here we assume 3PH wires all have tension t, singles t/3
        net = Vec2(0, 0)
        for d in wire_dirs:
            weight = 1.0 if n == 3 else (1/3)
            net += d * weight
        # equilibrium guy goes opposite net
        installs = [ -net.normalize() ]

    # 3) Insert new Down_Guy blocks at pole_pos + offset in those directions
    for dir_vec in installs:
        insert_pt = Vec2(pole_pos) + dir_vec * 5.0  # offset_distance = 5 drawing units; tweak as needed
        msp.add_blockref(
            down_guy_block_name,
            insert_pt,
            dxfattribs={
                "layer": "overhead_device",
                "xscale": scale,
                "yscale": scale,
                "rotation": math.degrees(math.atan2(dir_vec.y, dir_vec.x))
            }
        )

def replace_pole_with_library3(doc, output_path):
    msp = doc.modelspace()

    # ——— load pole & ground blocks as before ———
    # Removed_Pole_Block, Replaced_Pole_Block, Ground, etc.
    # … (your existing block-loading code here) …

    # Load new down-guy block once
    if "Down_Guy" not in doc.blocks:
        dg_doc = ezdxf.readfile("overhead/guy/Down_Guy.dxf")
        dg_block = doc.blocks.new(name="Down_Guy")
        for e in dg_doc.modelspace():
            dg_block.add_entity(e.copy())

    # Cache primary wires
    wires = [line for line in msp.query('LINE') if line.dxf.layer in primary_line_layers]

    for line in wires:
        # ——— your pole replacement + ground logic here ———
        # … (destroy poles, insert Removed/Replaced, insert Ground) …

        # After poles and text, handle down-guys at each connected pole:
        for pole in msp.query('INSERT[name ? "^Pole.*"]'):
            if is_close(pole.dxf.insert, line.dxf.start) or is_close(pole.dxf.insert, line.dxf.end):
                pole_pos = Vec2(pole.dxf.insert)
                # collect unit directions of all primary wires at this pole
                wire_dirs = []
                for w in wires:
                    if is_close(pole_pos, w.dxf.start):
                        vec = Vec2(w.dxf.end) - Vec2(w.dxf.start)
                        wire_dirs.append(vec.normalize())
                    elif is_close(pole_pos, w.dxf.end):
                        vec = Vec2(w.dxf.start) - Vec2(w.dxf.end)
                        wire_dirs.append(vec.normalize())
                # install new down-guys according to tension rules
                install_down_guys(msp, pole_pos, wire_dirs, scale=4.0)

    # … (rest of your text‐insertion code, save file, etc.) …
    doc.saveas(output_path)
    print(f"Saved modified DXF to {output_path}")


In [48]:
# Usage example:
doc = ezdxf.readfile("input.dxf")
replace_pole_with_library3(doc, "output.dxf")

Saved modified DXF to output.dxf


In [8]:
# Code where I try to combine the guy logic into the main code.
import math
import ezdxf
from ezdxf.math import Vec2

primary_line_layers = ['wire_3PH_primary', 'wire_X_primary', 'wire_Y_primary', 'wire_Z_primary', 'wire_2PH_primary']

def is_close(p1, p2, tol=1):
    print(Vec2(p1).distance(Vec2(p2)))
    return Vec2(p1).distance(Vec2(p2)) <= tol

def add_text_on_line(msp, start, end, text="RP", color=6, height=17):
    midpoint = Vec2(start).lerp(Vec2(end), factor=0.5)
    angle_rad = math.atan2(end[1] - start[1], end[0] - start[0])
    angle_deg = math.degrees(angle_rad)

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    # ✅ Set alignment to center on midpoint
    txt.dxf.insert = midpoint
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = midpoint


def add_length_label_below(msp, start, end, color=6, height= 13, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)
    
    length = start_v.distance(end_v)
    length_text = f"{length:.2f}"
    
    # Calculate angle of line in radians
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)
    
    # Vector pointing along the line
    line_dir = (end_v - start_v).normalize()
    # Perpendicular vector to the line (rotated 90 deg CCW)
    perp_dir = Vec2(-line_dir.y, line_dir.x)
    
    # Always offset downward visually means offset along negative perp_dir
    text_pos = midpoint - perp_dir * offset_distance * 20
    
    # Add the text
    txt = msp.add_text(
        length_text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos
    
# Can be used to put a text above line
def add_text_above_line(msp, start, end, text, color=6, height=10, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)

    # Angle of the line for text rotation
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)

    # Direction perpendicular to the line (90 degrees CCW)
    line_dir = (end_v - start_v).normalize()
    perp_dir = Vec2(-line_dir.y, line_dir.x)

    # Move in the **positive** perpendicular direction → above visually
    text_pos = midpoint + perp_dir * offset_distance * 15

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos


def install_down_guys(msp, pole_pos, wire_dirs, down_guy_block_name="Down_Guy", scale=1.0):
    """
    Remove existing down-guys at this pole, then install new ones to balance tension.
    wire_dirs: list of Vec2 unit-vectors pointing along each primary wire leaving the pole.
    """
    # 1) Remove old DownGuy blocks at this pole
    for guy in msp.query('INSERT[name=="DownGuy"]'):
        if guy.dxf.layer == "overhead_device" and is_close(pole_pos, guy.dxf.insert):
            guy.destroy()

    # 2) Compute equilibrium directions
    installs = []  # list of unit Vec2 directions where we'll place a down-guy
    n = len(wire_dirs)
    if n == 2:
        # two wires: if straight (180°), single guy opposite; else two guys opposite each wire
        angle = math.degrees(math.acos(wire_dirs[0].dot(wire_dirs[1])))
        if abs(angle - 180) < 1e-3:
            installs = [ -(wire_dirs[0] + wire_dirs[1]).normalize() ]
        else:
            installs = [ -d for d in wire_dirs ]
    elif n == 1:
        # one wire → place opposite it
        installs = [ -wire_dirs[0] ]
    else:
        # three or more: weight by tension (t for 3-phase, t/3 for singles) → net tension vector
        # here we assume 3PH wires all have tension t, singles t/3
        net = Vec2(0, 0)
        for d in wire_dirs:
            weight = 1.0 if n == 3 else (1/3)
            net += d * weight
        # equilibrium guy goes opposite net
        installs = [ -net.normalize() ]

    # 3) Insert new Down_Guy blocks at pole_pos + offset in those directions
    for dir_vec in installs:
        insert_pt = Vec2(pole_pos) + dir_vec * 5.0  # offset_distance = 5 drawing units; tweak as needed
        msp.add_blockref(
            down_guy_block_name,
            insert_pt,
            dxfattribs={
                "layer": "overhead_device",
                "xscale": scale,
                "yscale": scale,
                "rotation": math.degrees(math.atan2(dir_vec.y, dir_vec.x))
            }
        )


def replace_pole_with_library(doc, output_path):
    msp = doc.modelspace()

    # Load and define Removed_Pole_Block once
    if "Removed_Pole_Block" not in doc.blocks:
        removed_doc = ezdxf.readfile("Symbol List/DXF_Individual/pole/Removed Pole.dxf")
        removed_block = doc.blocks.new(name="Removed_Pole_Block")
        for e in removed_doc.modelspace():
            removed_block.add_entity(e.copy())

    # Load and define Replaced_Pole_Block once
    if "Replaced_Pole_Block" not in doc.blocks:
        replaced_doc = ezdxf.readfile("Symbol List/DXF_Individual/pole/Replaced Pole.dxf")
        replaced_block = doc.blocks.new(name="Replaced_Pole_Block")
        for e in replaced_doc.modelspace():
            replaced_block.add_entity(e.copy())

    # Load and define Ground block once
    if "Ground" not in doc.blocks:
        ground_doc = ezdxf.readfile("Symbol List/DXF_Individual/ground/Ground.dxf")
        ground_block = doc.blocks.new(name="Ground")
        for e in ground_doc.modelspace():
            ground_block.add_entity(e.copy())

        # Load new down-guy block once
    if "Down_Guy" not in doc.blocks:
        dg_doc = ezdxf.readfile("Symbol List/DXF_Individual/guy/Down_Guy.dxf")
        dg_block = doc.blocks.new(name="Down_Guy")
        for e in dg_doc.modelspace():
            dg_block.add_entity(e.copy())

    # Query existing Ground inserts on "overhead_device" layer for quick lookup
    existing_grounds = [e for e in msp.query('INSERT') if e.dxf.layer == "overhead_device" and e.dxf.name == "Ground"]

    # Cache primary wires
    wires = [line for line in msp.query('LINE') if line.dxf.layer in primary_line_layers]

    # Loop through wires
    for line in msp.query('LINE'):
        if line.dxf.layer in primary_line_layers:
            for pole in msp.query('INSERT[name ? "^New Pole.*"]'):
                pole_pos = pole.dxf.insert
                if is_close(pole_pos, line.dxf.start) or is_close(pole_pos, line.dxf.end):

                    ####### Down Guy Logic in main method  ######
                    # collect unit directions of all primary wires at this pole
                    wire_dirs = []
                    for w in wires:
                        if is_close(pole_pos, w.dxf.start):
                            vec = Vec2(w.dxf.end) - Vec2(w.dxf.start)
                            wire_dirs.append(vec.normalize())
                        elif is_close(pole_pos, w.dxf.end):
                            vec = Vec2(w.dxf.start) - Vec2(w.dxf.end)
                            wire_dirs.append(vec.normalize())
                    # install new down-guys according to tension rules
                    install_down_guys(msp, pole_pos, wire_dirs, scale=4.0)
                    #######  Down Guy installation part end  ########
                
                    ins_point = pole_pos
                    pole.destroy()  # Remove old pole

                    # Insert Removed and Replaced Poles (on "pole" layer)
                    msp.add_blockref("Removed_Pole_Block", ins_point, dxfattribs={"layer": "pole"})
                    msp.add_blockref("Replaced_Pole_Block", ins_point, dxfattribs={"layer": "pole"})

                    # Check if Ground block already exists at pole position (within tolerance)
                    ground_exists = False
                    for ground in existing_grounds:
                        if is_close(ins_point, ground.dxf.insert):
                            ground_exists = True
                            break

                    if not ground_exists:
                        # Insert Ground block scaled 4x on "overhead_device" layer
                        msp.add_blockref(
                            "Ground",
                            ins_point,
                            dxfattribs={
                                "layer": "overhead_device",
                                "xscale": 3.5,
                                "yscale": 3.5,
                            }
                        )

            # Add texts and labels
            add_text_on_line(msp, line.dxf.start, line.dxf.end)
            add_length_label_below(msp, line.dxf.start, line.dxf.end, offset_distance=1)
            add_text_above_line(msp, line.dxf.start, line.dxf.end, "This is test", offset_distance=1.5)

    # Update texts under pole_text layer
    pole_text = "pole_text"
    for txt in msp.query(f'*[layer=="{pole_text}"]'):
        old_content = txt.dxf.text
        new_content = f"{old_content}\n45ft-A"
        txt.dxf.text = new_content

    doc.saveas(output_path)
    print(f"Saved modified DXF to {output_path}")


# Usage example:
doc = ezdxf.readfile("Samples/unprocessed_dxf/input.dxf")
replace_pole_with_library(doc, "Samples/processed_dxf/output.dxf")
# Its working

0.0
0.0
195.92239079125451
404.86177461010857
404.86177461010857
834.1119614873659
834.1119614873659
1105.011859511728
834.1119614873659
926.9526808712994
1105.011859511728
1117.8161456495134
404.86177461010857
564.0685550133315
195.92239079125451
335.29927876597293
0.0
1105.011859511728
1294.2537917964032
195.92239079125451
0.0
195.92239079125451
0.0
0.0
244.12798641010897
670.902551515344
670.902551515344
939.551804082522
670.902551515344
742.5737928242238
939.551804082522
936.1297834926777
244.12798641010897
372.98373933290827
0.0
195.92239079125451
332.1200669636848
939.551804082522
1131.198493224747
335.29927876597293
180.6311066572102
404.86177461010857
244.12798641010897
834.1119614873659
670.902551515344
564.0685550133315
372.98373933290827
926.9526808712994
742.5737928242238
1105.011859511728
939.551804082522
1117.8161456495134
936.1297834926777
180.6311066572102
329.3713076192944
244.12798641010897
0.0
404.86177461010857
244.12798641010897
244.12798641010897
0.0
0.0
430.93424

In [4]:
# DWG <-> DXF converter

import subprocess
from pathlib import Path

def convert_dwg_to_dxf(input_file, output_folder, converter_exe):
    cmd = [
        str(converter_exe),
        str(Path(input_file).parent),
        str(output_folder),
        "ACAD2013",  # DXF version
        "DXF",
        "0",  # 0 = ASCII
        "1"   # 1 = include layouts
    ]
    subprocess.run(cmd, check=True)
    print(f"DWG → DXF converted: {Path(input_file).name}")

def convert_dxf_to_dwg(input_file, output_folder, converter_exe):
    cmd = [
        str(converter_exe),
        str(Path(input_file).parent),
        str(output_folder),
        "ACAD2013",  # DWG version
        "DWG",
        "0",  # irrelevant for DWG
        "1"   # 1 = include layouts
    ]
    subprocess.run(cmd, check=True)
    print(f"DXF → DWG converted: {Path(input_file).name}")


In [8]:
# Convert DXF → DWG
# Set these paths:
converter = Path(r"C:\Program Files\ODA\ODAFileConverter 26.4.0\ODAFileConverter.exe")

input_folder = Path(r"C:\Users\ravis\jupyter\Samples\processed_dxf")
output_folder = Path(r"C:\Users\ravis\jupyter\Samples\output_dwg")

output_folder.mkdir(parents=True, exist_ok=True)

for file in input_folder.glob("*.dxf"):
    convert_dxf_to_dwg(file, output_folder, converter)


DXF → DWG converted: output.dxf


In [6]:
# Convert DWG → DXF
# Set these paths:
converter = Path(r"C:\Program Files\ODA\ODAFileConverter 26.4.0\ODAFileConverter.exe")

input_folder = Path(r"C:\Users\ravis\jupyter\Samples\input_dwg")
output_folder = Path(r"C:\Users\ravis\jupyter\Samples\unprocessed_dxf")

output_folder.mkdir(parents=True, exist_ok=True)

for file in input_folder.glob("*.dwg"):
    convert_dwg_to_dxf(file, output_folder, converter)


DWG → DXF converted: input.dwg


In [2]:
# Bulk Converter

import subprocess
from pathlib import Path
import shutil

def convert_all_dxf_to_dwg_preserve_structure(input_root, output_root, converter_exe):
    input_root = Path(input_root).resolve()
    output_root = Path(output_root).resolve()
    
    for dxf_file in input_root.rglob("*.dxf"):
        # Determine relative path of the folder the DXF is in
        relative_folder = dxf_file.parent.relative_to(input_root)
        output_folder = output_root / relative_folder
        output_folder.mkdir(parents=True, exist_ok=True)
        
        # ODAFileConverter works at folder level, so convert the *parent folder* of this DXF
        subprocess.run([
            str(converter_exe),
            str(dxf_file.parent),
            str(output_folder),
            "ACAD2013",
            "DWG",
            "0",
            "1"
        ], check=True)
        
        # Rename the output file to match the original DXF name, but with .dwg
        generated_file = output_folder / (dxf_file.stem + ".dwg")
        print(f"Converted: {dxf_file.relative_to(input_root)} → {generated_file.relative_to(output_root)}")

# Example usage:
converter_exe = Path(r"C:\Program Files\ODA\ODAFileConverter 26.4.0\ODAFileConverter.exe")
input_folder = Path(r"C:\Users\ravis\jupyter\input_everything")
output_folder = Path(r"C:\Users\ravis\jupyter\output_everything")

convert_all_dxf_to_dwg_preserve_structure(input_folder, output_folder, converter_exe)


DWG → DXF converted: input.dwg


In [7]:
import subprocess
from pathlib import Path
import shutil

# Convert DWG → DXF
# Set these paths:
converter = Path(r"C:\Program Files\ODA\ODAFileConverter 26.4.0\ODAFileConverter.exe")

input_folder = Path(r"C:\Users\ravis\jupyter\Samples\input_dwg")
output_folder = Path(r"C:\Users\ravis\jupyter\Samples\unprocessed_dxf")

output_folder.mkdir(parents=True, exist_ok=True)

for file in input_folder.glob("*.dwg"):
    convert_dwg_to_dxf(file, output_folder, converter)

# get block data from single block
# Code where I try to combine the guy logic into the main code.
import math
import ezdxf
from ezdxf.math import Vec2

primary_line_layers = ['wire_3PH_primary', 'wire_X_primary', 'wire_Y_primary', 'wire_Z_primary', 'wire_2PH_primary']
block_library = ezdxf.readfile("Symbol List/DXF/combined_symbols_with_labels.dxf")

def is_close(p1, p2, tol=1):
    return Vec2(p1).distance(Vec2(p2)) <= tol

def add_text_on_line(msp, start, end, text="RP", color=6, height=17):
    midpoint = Vec2(start).lerp(Vec2(end), factor=0.5)
    angle_rad = math.atan2(end[1] - start[1], end[0] - start[0])
    angle_deg = math.degrees(angle_rad)

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    # ✅ Set alignment to center on midpoint
    txt.dxf.insert = midpoint
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = midpoint


def add_length_label_below(msp, start, end, color=6, height= 13, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)
    
    length = start_v.distance(end_v)
    length_text = f"{length:.2f}"
    
    # Calculate angle of line in radians
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)
    
    # Vector pointing along the line
    line_dir = (end_v - start_v).normalize()
    # Perpendicular vector to the line (rotated 90 deg CCW)
    perp_dir = Vec2(-line_dir.y, line_dir.x)
    
    # Always offset downward visually means offset along negative perp_dir
    text_pos = midpoint - perp_dir * offset_distance * 20
    
    # Add the text
    txt = msp.add_text(
        length_text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos
    
# Can be used to put a text above line
def add_text_above_line(msp, start, end, text, color=6, height=10, offset_distance=2.0):
    start_v = Vec2(start)
    end_v = Vec2(end)
    midpoint = start_v.lerp(end_v, 0.5)

    # Angle of the line for text rotation
    angle_rad = math.atan2(end_v.y - start_v.y, end_v.x - start_v.x)
    angle_deg = math.degrees(angle_rad)

    # Direction perpendicular to the line (90 degrees CCW)
    line_dir = (end_v - start_v).normalize()
    perp_dir = Vec2(-line_dir.y, line_dir.x)

    # Move in the **positive** perpendicular direction → above visually
    text_pos = midpoint + perp_dir * offset_distance * 15

    txt = msp.add_text(
        text,
        dxfattribs={
            "height": height,
            "color": color,
            "rotation": angle_deg,
            "layer": "annotations",
        }
    )
    txt.dxf.insert = text_pos
    txt.dxf.halign = 1  # CENTER
    txt.dxf.valign = 2  # MIDDLE
    txt.dxf.align_point = text_pos


def install_down_guys(msp, pole_pos, wire_dirs, down_guy_block_name="Down Guy", scale=1.0):
    """
    Remove existing down-guys at this pole, then install new ones to balance tension.
    wire_dirs: list of Vec2 unit-vectors pointing along each primary wire leaving the pole.
    """
    # 1) Remove old DownGuy blocks at this pole
    for guy in msp.query('INSERT[name=="Down Guy"]'):
        if guy.dxf.layer == "overhead_device" and is_close(pole_pos, guy.dxf.insert):
            guy.destroy()

    # 2) Compute equilibrium directions
    installs = []  # list of unit Vec2 directions where we'll place a down-guy
    n = len(wire_dirs)
    if n == 2:
        # Calculate angle between the two wires
        angle = math.degrees(math.acos(max(-1, min(1, wire_dirs[0].dot(wire_dirs[1])))))

        if abs(angle - 180) <= 3:
            # Wires are nearly perfectly opposite → equilibrium → NO guys needed
            installs = []
        else:
            # If the *smaller* angle < 145°, need two guys, opposite to each wire
            small_angle = min(angle, 360 - angle)
            if small_angle < 145:
                installs = [-wire_dirs[0], -wire_dirs[1]]
            else:
                # Otherwise, use single guy opposite to resultant (equilibrium vector)
                resultant = (wire_dirs[0] + wire_dirs[1])
                if resultant.magnitude != 0:
                    installs = [-resultant.normalize()]
                else:
                    # Degenerate case: resultant = 0 → fallback to using two opposite guys
                    installs = [-wire_dirs[0], -wire_dirs[1]]
    elif n == 1:
        # one wire → place opposite it
        installs = [ -wire_dirs[0] ]
    else:
        # three or more: weight by tension (t for 3-phase, t/3 for singles) → net tension vector
        # here we assume 3PH wires all have tension t, singles t/3
        net = Vec2(0, 0)
        for d in wire_dirs:
            weight = 1.0 if n == 3 else (1/3)
            net += d * weight
        # equilibrium guy goes opposite net
        installs = [ -net.normalize() ]

    # 3) Insert new Down_Guy blocks at pole_pos + offset in those directions
    for dir_vec in installs:
        insert_pt = Vec2(pole_pos) + dir_vec * 5.0  # offset_distance = 5 drawing units; tweak as needed
        msp.add_blockref(
            down_guy_block_name,
            insert_pt,
            dxfattribs={
                "layer": "overhead_device",
                "xscale": scale,
                "yscale": scale,
                "rotation": math.degrees(math.atan2(dir_vec.y, dir_vec.x))
            }
        )


# Load the block library once (shared file for all blocks)
def add_block_from_library(doc, block_name, new_name=None):
    new_name = new_name or block_name
    if new_name in doc.blocks:
        return  # Block already defined in target doc

    try:
        library_block = block_library.blocks[block_name]
    except KeyError:
        print(f"Block '{block_name}' not found in block library.")
        return

    new_block = doc.blocks.new(name=new_name)
    for e in library_block:
        new_block.add_entity(e.copy())


def replace_pole_with_library(doc, output_path):
    msp = doc.modelspace()

    # Usage in your code:
    add_block_from_library(doc, "Replaced Pole", new_name="Replaced Pole")
    add_block_from_library(doc, "Removed Pole", new_name="Removed Pole")
    add_block_from_library(doc, "Ground", new_name="Ground")
    add_block_from_library(doc, "Down Guy", new_name="Down Guy")

    # Query existing Ground inserts on "overhead_device" layer for quick lookup
    existing_grounds = [e for e in msp.query('INSERT') if e.dxf.layer == "overhead_device" and e.dxf.name == "Ground"]

    # Cache primary wires
    wires = [line for line in msp.query('LINE') if line.dxf.layer in primary_line_layers]

    # Loop through wires
    for line in msp.query('LINE'):
        if line.dxf.layer in primary_line_layers:
            for pole in msp.query('INSERT[name ? "^New Pole.*"]'):
                pole_pos = pole.dxf.insert
                if is_close(pole_pos, line.dxf.start) or is_close(pole_pos, line.dxf.end):

                    ####### Down Guy Logic in main method  ######
                    # collect unit directions of all primary wires at this pole
                    wire_dirs = []
                    for w in wires:
                        if is_close(pole_pos, w.dxf.start):
                            vec = Vec2(w.dxf.end) - Vec2(w.dxf.start)
                            wire_dirs.append(vec.normalize())
                        elif is_close(pole_pos, w.dxf.end):
                            vec = Vec2(w.dxf.start) - Vec2(w.dxf.end)
                            wire_dirs.append(vec.normalize())
                    # install new down-guys according to tension rules
                    install_down_guys(msp, pole_pos, wire_dirs)
                    #######  Down Guy installation part end  ########
                
                    ins_point = pole_pos
                    pole.destroy()  # Remove old pole

                    # Insert Removed and Replaced Poles (on "pole" layer)
                    msp.add_blockref("Removed Pole", ins_point, dxfattribs={"layer": "pole"})
                    msp.add_blockref("Replaced Pole", ins_point, dxfattribs={"layer": "pole"})

                    # Check if Ground block already exists at pole position (within tolerance)
                    ground_exists = False
                    for ground in existing_grounds:
                        if is_close(ins_point, ground.dxf.insert):
                            ground_exists = True
                            break

                    if not ground_exists:
                        # Insert Ground block scaled 4x on "overhead_device" layer
                        msp.add_blockref(
                            "Ground",
                            ins_point,
                            dxfattribs={
                                "layer": "overhead_device",
                            }
                        )

            # Add texts and labels
            add_text_on_line(msp, line.dxf.start, line.dxf.end)
            add_length_label_below(msp, line.dxf.start, line.dxf.end, offset_distance=1)
            add_text_above_line(msp, line.dxf.start, line.dxf.end, "This is test", offset_distance=1.5)

    # Update texts under pole_text layer
    pole_text = "pole_text"
    for txt in msp.query(f'*[layer=="{pole_text}"]'):
        old_content = txt.dxf.text
        new_content = f"{old_content}\n45ft-A"
        txt.dxf.text = new_content

    doc.saveas(output_path)
    print(f"Saved modified DXF to {output_path}")


# Usage example:
doc = ezdxf.readfile("Samples/unprocessed_dxf/input.dxf")
replace_pole_with_library(doc, "Samples/processed_dxf/output.dxf")

# Converting it back to te DWG file.
# Convert DWG → DXF
# Set these paths:
input_folder = Path(r"C:\Users\ravis\jupyter\Samples\processed_dxf")
output_folder = Path(r"C:\Users\ravis\jupyter\Samples\output_dwg")

output_folder.mkdir(parents=True, exist_ok=True)

for file in input_folder.glob("*.dxf"):
    convert_dwg_to_dxf(file, output_folder, converter)


DWG → DXF converted: input.dwg
Saved modified DXF to Samples/processed_dxf/output.dxf
DWG → DXF converted: output.dxf
